# 🚀 Iteracja 1: Podstawowe promptowanie (Zero-shot)

**Czas:** ~35 minut  
**Poziom:** Wejściowy

---

## Cel iteracji

Nauczyć się wysyłać zapytania do modelu językowego przez API i klasyfikować recenzje przy użyciu **zero-shot promptowania** - bez podawania przykładów.

---

## Teoria: Anatomia promptu

Prompt to instrukcja, którą wysyłamy do modelu. Składa się z dwóch kluczowych części:

| Rola | Opis | Kiedy używać |
|------|------|--------------|
| `system` | Definiuje rolę i zasady modelu | Instrukcje ogólne, persona, ograniczenia |
| `user` | Konkretne zadanie do wykonania | Dane do przetworzenia, pytania |

**Dlaczego rozdzielamy `system` od `user`?**

- Separacja odpowiedzialności - `system` to "kto jest model", `user` to "co ma zrobić"
- Pozwala na łatwe reuznanie tej samej konfiguracji modelu z różnymi danymi wejściowymi
- Modele są trenowane z tym podziałem - daje lepsze wyniki

```python
# ❌ ZŁE: Mieszanie roli i zadania
{"role": "user", "content": "Jesteś ekspertem. Sklasyfikuj tekst."}

# ✅ DOBRE: Separacja
{"role": "system", "content": "Jesteś ekspertem od analizy tekstu."}
{"role": "user", "content": "Sklasyfikuj poniższy tekst."}
```

---

## Zero-shot promptowanie

**Zero-shot** = model klasyfikuje bez żadnych przykładów. Polegamy wyłącznie na wiedzy modelu i jakości naszych instrukcji.

**Zalety:** Prosto, szybko, nie wymaga przygotowania przykładów  
**Wady:** Mniejsza dokładność, model może interpretować kategorie po swojemu

## ⚙️ Setup - środowisko

Ta komórka wykrywa czy jesteś w **Google Colab** czy lokalnie, a następnie:
1. Klonuje repozytorium z GitHub (tylko Colab)
2. Instaluje wymagane biblioteki
3. Konfiguruje ścieżki importów

In [ ]:
import sys
import subprocess
from pathlib import Path

# Wykryj środowisko
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

print(f"{'🟡 Google Colab' if IN_COLAB else '💻 Lokalne środowisko'}")

# ============================================================
# KONFIGURACJA REPO (zmień URL na właściwe przed warsztatami)
# ============================================================
REPO_URL = "https://github.com/JSerek/techland-genai-workshop.git"
REPO_DIR = Path("/content/szkolenie_techland") if IN_COLAB else Path(".").resolve().parent
# ============================================================

if IN_COLAB:
    if not REPO_DIR.exists():
        print(f"📥 Klonuję repo...")
        subprocess.run(["git", "clone", REPO_URL, str(REPO_DIR)], check=True)
        print("✅ Repo sklonowane")
    else:
        print("🔄 Aktualizuję repo (git pull)...")
        subprocess.run(["git", "-C", str(REPO_DIR), "pull"], check=True)

    # Dodaj repo do sys.path
    sys.path.insert(0, str(REPO_DIR))

    # Zainstaluj zależności
    print("📦 Instaluję biblioteki...")
    subprocess.run(
        [sys.executable, "-m", "pip", "install", "-r", str(REPO_DIR / "requirements.txt"), "-q"],
        check=True
    )
    print("✅ Biblioteki zainstalowane")
else:
    # Lokalnie: dodaj root projektu do sys.path
    sys.path.insert(0, str(REPO_DIR))
    print(f"📂 Ścieżka projektu: {REPO_DIR}")

print("\n✅ Setup zakończony")

In [ ]:
# ============================================================
# 🔑 Konfiguracja API
# W Google Colab: dodaj sekrety w menu po lewej (🔑 ikona)
#   Nazwy sekretów: VERTEX_AI_API_KEY, VERTEX_AI_BASE_URL, MODEL_NAME
#
# Lokalnie: ustaw zmienne środowiskowe lub wpisz wartości bezpośrednio
# ============================================================

if IN_COLAB:
    from google.colab import userdata
    API_KEY   = userdata.get("VERTEX_AI_API_KEY")
    BASE_URL  = userdata.get("VERTEX_AI_BASE_URL")
    MODEL_NAME = userdata.get("MODEL_NAME") or "google/gemini-2.5-flash-lite"
else:
    import os
    API_KEY   = os.environ.get("VERTEX_AI_API_KEY", "TODO: wklej API key")
    BASE_URL  = os.environ.get("VERTEX_AI_BASE_URL", "TODO: wklej endpoint URL")
    MODEL_NAME = os.environ.get("MODEL_NAME", "google/gemini-2.5-flash-lite")

# Walidacja
if not API_KEY or API_KEY == "TODO: wklej API key":
    print("❌ Brak API_KEY! Dodaj sekret 'VERTEX_AI_API_KEY' w Colab Secrets.")
elif not BASE_URL or BASE_URL == "TODO: wklej endpoint URL":
    print("❌ Brak BASE_URL! Dodaj sekret 'VERTEX_AI_BASE_URL' w Colab Secrets.")
else:
    print(f"✅ API skonfigurowane: {MODEL_NAME}")
    print(f"   Endpoint: {BASE_URL[:60]}...")

In [ ]:
# Konfiguracja klienta LLM - dane pobrane z Secrets w komórce powyżej
from src.utils.llm_client import create_client, LLMProvider

PROVIDER = LLMProvider.VERTEX_AI  # Zmień jeśli używasz innego providera

client = create_client(
    provider=PROVIDER,
    api_key=API_KEY,
    base_url=BASE_URL,
)
print(f"✅ Klient LLM gotowy: {PROVIDER.value} / {MODEL_NAME}")

## 📂 Ładowanie danych

In [ ]:
import json
import pandas as pd
from pathlib import Path

# Ścieżki do danych (względem root repo)
DATA_DIR = REPO_DIR / "data"

# Próbka recenzji
df = pd.read_csv(DATA_DIR / "processed" / "workshop_sample.csv")
review_texts = df["review_text"].tolist()
print(f"✅ Załadowano {len(review_texts)} recenzji")

# Golden dataset
golden_path = DATA_DIR / "evaluation" / "golden_dataset.json"
if golden_path.exists():
    with open(golden_path) as f:
        golden_data = json.load(f)
    golden_texts = [item["review_text"] for item in golden_data]
    golden_labels = [item["labels"] for item in golden_data]
    print(f"✅ Golden dataset: {len(golden_data)} przykładów")
else:
    golden_texts, golden_labels = [], []
    print("⚠️  Golden dataset nie znaleziony - ewaluacja niedostępna")

In [ ]:
# Wczytaj golden dataset (jeśli gotowy)
# ============================================================
# TODO: Uzupełnij po przygotowaniu golden dataset
# ============================================================
GOLDEN_DATASET_PATH = "../data/evaluation/golden_dataset.json"  # TODO

golden_path = Path(GOLDEN_DATASET_PATH)
if golden_path.exists():
    with open(golden_path) as f:
        golden_data = json.load(f)
    golden_texts = [item["review_text"] for item in golden_data]
    golden_labels = [item["labels"] for item in golden_data]
    print(f"✅ Golden dataset: {len(golden_data)} przykładów")
else:
    print("⚠️  Golden dataset nie znaleziony - ewaluacja niedostępna")
    golden_texts, golden_labels = [], []

## 🎯 Zadanie: Napisz prompt klasyfikujący

### Twoje zadanie:

Napisz prompt, który klasyfikuje recenzję gry do jednej lub więcej kategorii tematycznych.

> 🚧 **TODO:** Uzupełnij kategorie po analizie danych

**Wskazówki do dobrego promptu:**
- Podaj dokładny format odpowiedzi (co model ma zwrócić)
- Zdefiniuj kategorie precyzyjnie - model musi wiedzieć co oznaczają
- Ogranicz model tylko do zdefiniowanych kategorii
- Testuj na kilku przykładach zanim puścisz na całym zbiorze

In [ ]:
# ============================================================
# KONFIGURACJA kategorii
# TODO: Uzupełnij po analizie danych
# ============================================================
CATEGORIES = [
    # Przykładowy format - zmień na właściwe kategorie:
    # "performance",    # Problemy z wydajnością, FPS, loading
    # "bug",            # Błędy, crashe, glitche
    # "story",          # Fabuła, narracja, postacie
    # "gameplay",       # Mechaniki rozgrywki, sterowanie
    # "graphics",       # Grafika, oprawa wizualna
    # "audio",          # Muzyka, dźwięki, dubbing
    # "price",          # Cena, wartość za pieniądze
    # "other",          # Inne
]

categories_str = ", ".join(f'"{c}"' for c in CATEGORIES)
print(f"Dostępne kategorie: {categories_str or '⚠️  Brak - uzupełnij CATEGORIES'}")

In [ ]:
# ============================================================
# 🖊️  ĆWICZENIE: Napisz swój prompt
# Zmodyfikuj poniższy szablon - zmień treść, dodaj/usuń instrukcje
# ============================================================

SYSTEM_PROMPT = """Jesteś ekspertem od analizy recenzji gier komputerowych.
Twoim zadaniem jest klasyfikacja tematyczna recenzji.

Dostępne kategorie: {categories}

Zasady:
- Możesz przypisać jedną lub więcej kategorii
- Używaj tylko kategorii z powyższej listy
- Odpowiedz WYŁĄCZNIE listą kategorii oddzielonych przecinkami, bez żadnego komentarza

Przykład formatu odpowiedzi: bug, performance""".format(categories=categories_str)

USER_PROMPT_TEMPLATE = """Sklasyfikuj poniższą recenzję gry:

Recenzja: {review_text}

Kategorie:"""

print("SYSTEM PROMPT:")
print(SYSTEM_PROMPT)
print("\nUSER PROMPT TEMPLATE:")
print(USER_PROMPT_TEMPLATE)

## 🧪 Test na jednej recenzji

Zawsze najpierw przetestuj na jednym przykładzie!

In [ ]:
def classify_review(review_text: str) -> list[str]:
    """Klasyfikuje recenzję do kategorii tematycznych."""
    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": USER_PROMPT_TEMPLATE.format(review_text=review_text)},
        ],
        temperature=0.0,  # 0 = deterministyczny output (lepsza reprodukowalność)
    )

    raw_output = response.choices[0].message.content.strip()

    # Parsuj odpowiedź: "bug, performance" → ["bug", "performance"]
    labels = [label.strip().lower() for label in raw_output.split(",") if label.strip()]
    return labels


# Test na pierwszej recenzji
test_review = review_texts[0]
print(f"Recenzja: '{test_review[:300]}...'")
print()

result = classify_review(test_review)
print(f"Klasyfikacja: {result}")

## 📊 Klasyfikacja całego zbioru

Uruchom klasyfikację na całym golden dataset (lub próbce).

In [ ]:
from tqdm.notebook import tqdm

# Użyj golden dataset (lub próbki jeśli golden nie gotowy)
texts_to_classify = golden_texts if golden_texts else review_texts[:10]

print(f"Klasyfikuję {len(texts_to_classify)} recenzji...")

predictions = []
errors = []

for i, text in enumerate(tqdm(texts_to_classify)):
    try:
        labels = classify_review(text)
        predictions.append(labels)
    except Exception as e:
        print(f"Błąd dla recenzji {i}: {e}")
        predictions.append([])  # pusta lista = brak klasyfikacji
        errors.append(i)

print(f"\n✅ Klasyfikacja zakończona. Błędy: {len(errors)}/{len(texts_to_classify)}")

## 📈 Ewaluacja wyników

In [ ]:
if golden_labels:
    trial1 = evaluate_trial(
        trial_name=f"{MODEL_NAME} + zero-shot v1",
        model=MODEL_NAME,
        prompt_variant="zero-shot v1",
        predictions=predictions,
        expected=golden_labels,
        review_texts=golden_texts,
        strategy=MatchStrategy.CONTAINS_ALL,
    )

    # Wyświetl wyniki
    trial1.display()

    # Zapisz wyniki do porównania w kolejnych iteracjach
    print(f"\n💾 Accuracy Iteracja 1: {trial1.summary.accuracy:.1%}")
else:
    print("⚠️  Golden dataset nie jest gotowy - pokazuję tylko predykcje:")
    for text, pred in zip(texts_to_classify[:5], predictions[:5]):
        print(f"  Recenzja: '{text[:100]}...'")
        print(f"  Kategorie: {pred}")
        print()

## 💡 Ćwiczenie dla uczestników

### Zadanie:
Zmodyfikuj `SYSTEM_PROMPT` i `USER_PROMPT_TEMPLATE` tak, żeby poprawić accuracy klasyfikacji.

**Pomysły do eksperymentowania:**
1. Dodaj definicje kategorii (co dokładnie należy do każdej)
2. Zmień format odpowiedzi (np. JSON zamiast CSV)
3. Dodaj instrukcję "If unsure, use 'other'"
4. Zmień język promptu (angielski vs polski)
5. Dodaj/usuń przykłady formatu odpowiedzi

**Pytania do refleksji:**
- Które recenzje model klasyfikuje najgorzej?
- Czy model nadaje zbyt wiele kategorii, czy za mało?
- Jak wpływa zmiana temperatury (0.0 → 0.7) na wyniki?

In [ ]:
# Miejsce na Twój eksperyment - skopiuj i zmodyfikuj prompty powyżej
MY_SYSTEM_PROMPT = """
# TODO: Wpisz swój system prompt tutaj
"""

MY_USER_PROMPT_TEMPLATE = """
# TODO: Wpisz swój user prompt tutaj
"""

# Gdy gotowy - uruchom klasyfikację i ewaluację:
# ...
print("Zmodyfikuj prompty powyżej i uruchom klasyfikację!")

---

## ✅ Podsumowanie Iteracji 1

**Czego się nauczyłeś:**
- Struktura promptu: `system` + `user` i dlaczego to ważne
- Zero-shot classificationvia API
- Parsowanie surowego stringa z odpowiedzią modelu
- Ewaluacja accuracy z golden dataset

**Problem do rozwiązania w Iteracji 2:**  
Surowy string z modelu jest trudny do parsowania i może mieć różne formaty.  
→ Rozwiązanie: **Structured Output z Pydantic**